In [6]:
from sage.algebras.flag_algebras import *

In [7]:
from scipy.optimize import linprog
c = [-1, 4]
A = [[-3, 1], [1, 2]]
b = [6, 4]
x0_bounds = (None, None)
x1_bounds = (-3, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds], method="highs-ds")
res

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -22.0
              x: [ 1.000e+01 -3.000e+00]
            nit: 0
          lower:  residual: [       inf  0.000e+00]
                 marginals: [ 0.000e+00  6.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 3.900e+01  0.000e+00]
                 marginals: [-0.000e+00 -1.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [219]:
#rounding based on ldl decomposition and continued fractions

#continued fraction rounding
def cfr(value, accuracy=1e3):
    cf = continued_fraction(value)
    for ii, xx in enumerate(cf.quotients()):
        if xx>=accuracy:
            if ii>0:
                return cf.convergent(ii-1)
            return 0
    return cf.value()

#rounding for lists
def cfr_list(ls, accuracy=1e3):
    return [cfr(xx, accuracy) for xx in ls]

#rounding for matrices
def cfr_matrix(mat, accuracy=1e3):
    return [cfr_list(xx, accuracy) for xx in mat]

#rounding matrix based on LDL factoring
def cfr_ldl(mat, accuracy=1e3):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, cfr_matrix(mat_ldl[1], accuracy))
    D = diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), accuracy))
    return P*L*D*L.T*P.T

#rounding matrix based on LDL factoring with given kernel
def cfr_ldl_corr(mat, kern, accuracy=1e3):
    # mat is a floating point symmetric positive semidefinite matrix
    # kern is a rational symmetric positive semidefinite matrix

    print("mat input: \n", matrix(mat), "\n\n\nkern input: \n", matrix(kern), "\n\n\n")
    
    mat_ldl = matrix(mat).block_ldlt()
    kern_ldl = matrix(kern).block_ldlt()
    
    P_mat = matrix(QQ, mat_ldl[0])
    L_mat = P_mat * matrix(QQ, cfr_matrix(mat_ldl[1], accuracy)) * P_mat.T
    D_mat = P_mat * diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), accuracy)) * P_mat.T
    
    P_kern = kern_ldl[0]
    L_kern = P_kern * kern_ldl[1] * P_kern.T
    D_kern = P_kern * kern_ldl[2] * P_kern.T
    
    print("mat LD: \n", L_mat, "\n\n", D_mat, "\n\n\nkern LD: \n", L_kern, "\n\n", D_kern, "\n\n\n")
    
    # Get nonzero elements from kern and mat's L values
    kern_nonzero_L = [L_kern.T[ii] for ii in range(D_kern.nrows()) if D_kern[ii, ii] != 0]
    mat_nonzero_inds = [jj for jj in range(D_mat.nrows()) if D_mat[jj, jj] != 0]

    print("kern nonzero Ls: \n", kern_nonzero_L, "\n\n\nmat nonzero inds: \n", mat_nonzero_inds)
    
    # Adjust L_mat based on kern's constraints
    for jj in mat_nonzero_inds:
        mjj = L_mat.T[jj]
        gmb = matrix(QQ, kern_nonzero_L + [mjj])
        Ggmb, Mgmb = gmb.gram_schmidt()
        L_mat.T[jj] = Ggmb[-1]

    print("rounded mat L is: \n", L_mat)
    
    # Reconstruct the matrix
    mat_rounded = L_mat * D_mat * L_mat.T
    
    return mat_rounded

In [220]:
cfr_ldl_corr(MAT[0], KERN[0])

mat input: 
 [   0.4911025973196321   -0.2993373449763204   -0.3234061319078033   0.06585329730683441  -0.10404153488210859  -0.18037340909410096]
[  -0.2993373449763204   0.27414370746164235   0.02321498894643019  -0.08718987875574997   0.11508368287115034   0.08151706236552436]
[  -0.3234061319078033   0.02321498894643019    0.5820599175440999   0.04971745843266741  0.002133180157467668   0.05496727513862353]
[  0.06585329730683441  -0.08718987875574997   0.04971745843266741   0.18681853493081696  -0.11696566478584502 -0.021132858090005018]
[ -0.10404153488210859   0.11508368287115034  0.002133180157467668  -0.11696566478584502   0.38289317290769675  -0.07265235479567703]
[ -0.18037340909410096   0.08151706236552436   0.05496727513862353 -0.021132858090005018  -0.07265235479567703    0.4282362719330803] 


kern input: 
 [ 9/64  9/64 9/128     0     0 3/128]
[ 9/64  9/64 9/128     0     0 3/128]
[9/128 9/128 9/256     0     0 3/256]
[    0     0     0     0     0     0]
[    0     0  

[                                                                                                                                                                                    217740687/443371076                                                                                                                                                                     -8833718260422445/29510912716624952                                                                                                                                                                      -3209546282553201/9924197366388152                                                                                                                                                                      3474783471410801/52765519928645688                                                                                                                                                                     -5243328201877925/50396490284569892

In [183]:
MAT = []
KERN = []

In [185]:
def optimize_problem(self, target_element, target_size, maximize=True, \
                     ftypes=None, positives=None, \
                     certificate=False, rounding=None):
    r"""
    Try to maximize or minimize the value of `target_element`
    
    The algorithm calculates the multiplication tables and 
    sends the SDP problem to cvxopt.
    
    INPUT:

    - ``target_element`` -- Flag or FlagAlgebraElement; 
        the target whose density this function tries to
        maximize or minimize in large structures.
    - ``target_size`` -- integer; The program evaluates
        flags and the relations between them up to this
        size.
    - ``maximize`` -- boolean (default: `True`); 
    - ``ftypes`` -- list of ftypes (default: `None`); 
        the list of ftypes the program will consider,
        if not provided (or `None`) then generates all
        possible ftypes.
    - ``positives`` -- list of flag algebra elements, 
        optimizer will assume those are positive, can
        have different types.
    - ``certificate`` -- boolean (default: `False`);
        should it return a certificate of the bound.
    - ``rounding`` -- (default: `None`); if None, no 
        rounding, otherwise this specifies the accuracy

    OUTPUT: A bound for the optimization problem. If 
        certificate is requested then returns a density
        of the limit structure and the SOS proof matrices

    EXAMPLES::
    
    
    Mantel's theorem, calculate the maximum density
    of edges in triangle-free graphs ::

        sage: from sage.algebras.flag_algebras import *
        sage: GraphTheory.exclude(GraphTheory(3, edges=[[0, 1], [0, 2], [1, 2]]))
        sage: x = GraphTheory.optimize_problem(GraphTheory(2, edges=[[0, 1]]), 3)
        ...
        sage: abs(x-0.5)<1e-6
        True
    
    Generalized Turan problem, for example maximum density of K_3
    in K_5 -free graphs. The complement is calculated, optimizing empty
    E_3 in E_5 -free graphs. They are equivalent

        sage: GraphTheory.exclude(GraphTheory(5))
        sage: x = GraphTheory.optimize_problem(GraphTheory(3), 5) # long time (5 second)
        ...
    
    
    Generalized Turan problem for hypergraphs, maximum density of K^3_4
    in K^3_5 -free 3-graphs. Again, the complement is calculated. This is 
    long and should not be normally tested

        sage: ThreeGraphTheory.exclude(ThreeGraphTheory(5))
        sage: ThreeGraphTheory.optimize_problem(ThreeGraphTheory(4), 6) # todo: not implemented
        ...
    
    
    The minimum number of transitive tournaments is attained at 
    a random tournament [CoRa2015]_::

        sage: x = TournamentTheory.optimize_problem( \
        ....: TournamentTheory(3, edges=[[0, 1], [0, 2], [1, 2]]), \
        ....: 3, maximize=False)
        ...
    
    
    Ramsey's theorem, the K_5 is the largest 2-colorable complete
    graph without monochromatic K_3. (see [LiPf2021]_) ::

        sage: RamseyGraphTheory.exclude(RamseyGraphTheory(3, edges=[[0, 1], [0, 2], [1, 2]]))
        sage: x = RamseyGraphTheory.optimize_problem(RamseyGraphTheory(2), 4, maximize=False)
        ...
        sage: abs(x-0.2)<1e-6
        True
    
    .. NOTE::

        If `target_size` is too large, the calculation might take a 
        really long time. On a personal computer the following 
        maximum target_size values are recommended:
        -GraphTheory: 8
        -ThreeGraphTheory: 6
        -DiGraphTheory: 5
        -TournamentTheory: 9
        -PermutationTheory: 8
        -RamseyGraphTheory: 8
        -OVGraphTheory: 5
        -OEGraphTheory: 4
    """
    import time
    import sys

    try:
        from csdpy import solve_sdp
    except:
        print("No csdpy solver, can't run the optimizer!")
        return
    
    current = time.time()
    #calculate constraints from positive vectors
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
        constraints_vals = [0]*len(constraints_flags)
    
    #calculate ftypes
    if ftypes is None:
        flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
                  for flag in self.generate_flags(kk)]
        ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
                  for flag in flags]

    print("Ftypes constructed in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    block_sizes = [len(self.generate_flags((target_size + \
                   ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])

    block_sizes.extend([-constraints, -len(constraints_vals)])
    block_num = len(block_sizes)
    mat_inds = []
    mat_vals = []
    print("Block sizes done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    print("Block sizes are {}".format(block_sizes), flush=True)
    print("Calculating product matrices for {} ftypes and {} structures".format(len(ftypes), constraints), flush=True)
    
    pbar = None
    has_tqdm = True
    try:
        from tqdm import tqdm
        pbar = tqdm(enumerate(ftypes), file=sys.stdout)
    except:
        pbar = enumerate(ftypes)
        has_tqdm = False
    
    for ii, ftype in pbar:
        ns = (target_size + ftype.size())//2
        fls = self.generate_flags(ns, ftype)
        table = self.mul_project_table(ns, ns, ftype, [])
        for gg, mm in enumerate(table):
            dd = mm._dict()
            if len(dd)>0:
                inds, values = zip(*mm._dict().items())
                iinds, jinds = zip(*inds)
                for cc in range(len(iinds)):
                    if iinds[cc]>=jinds[cc]:
                        mat_inds.extend([gg+1, ii+1, iinds[cc]+1, 
                                         jinds[cc]+1])
                        mat_vals.append(values[cc])
        if has_tqdm: 
            pbar.set_description("{} is complete".format(ftype))
        else:
            print("{} is complete".format(ftype), flush=True)
    
    print("Table calculation done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    if maximize:
        avals = (target_element.project()*(-1)<<(target_size - \
                                       target_element.size())).values()
    else:
        avals = (target_element.project()<<(target_size - \
                                  target_element.size())).values()

    for ii in range(len(constraints_vals)):
        mat_inds.extend([0, block_num, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    constraints_flags_vec = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    for gg in range(constraints):
        mat_inds.extend([gg+1, block_num-1, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(len(constraints_flags_vec)):
            mat_inds.extend([gg+1, block_num, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    print("Target and constraint calculation done in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()
    
    sdp_result = solve_sdp(block_sizes, list(avals), 
                           mat_inds, mat_vals)
    print("SDP finished in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()

    # check scipy is installed, so we can do LP for the rounding
    if rounding != None:
        try:
            from scipy.optimize import linprog
        except:
            print("SciPy is needed for the rounding!")
            rounding=None
    
    if rounding==None:
        #No rounding, just return the result
        if maximize:
            ret = max(-sdp_result['primal'], -sdp_result['dual'])
        else:
            ret = min(sdp_result['primal'], sdp_result['dual'])
        print("Result is {}".format(ret), flush=True)
        if certificate:
            ralg = FlagAlgebra(RR, self)
            vec = ralg(target_size, sdp_result['y'])
            ret = (ret, vec, sdp_result)
        return ret
    else:
        #
        # Rounding code
        #
        
        accuracy = 10**max(rounding, 2)
        
        #First get a good rounded y:
        print("Rounding dual", flush=True); current = time.time()
        yvec = vector(sdp_result['y'])
        onevec = one_vector.values()
        
        best_y = None
        best_err = 1000
        
        for ex in range(1, 8):
            ry = vector(cfr_list(yvec, 10**ex))
            prod = onevec*ry
            
            if prod != 0 and abs(prod-1)<best_err:
                best_err = abs(prod-1)
                best_y = ry/prod
        
        #Then loop through the tables, keeping track the y-corrected and
        #normal slack values

        slack_base = vector(list(avals))
        slack_corr = vector(list(avals))
        
        correct_y = True
        
        pbar = None
        if has_tqdm:
            pbar = tqdm(enumerate(ftypes), file=sys.stdout)
        else:
            pbar = enumerate(ftypes)
        
        for ii, ftype in pbar:
            #calculation of the table entries
            ns = (target_size + ftype.size())//2
            fls = self.generate_flags(ns, ftype)
            table = self.mul_project_table(ns, ns, ftype, [])
            
            #if still correct, calculate the Z matrix
            if correct_y:
                Zii = None
                for jj, mat in enumerate(table):
                    if Zii==None:
                        Zii = mat*best_y[jj]
                    else:
                        Zii += mat*best_y[jj]
                if min(Zii.eigenvalues())<0:
                    correct_y = False

            #the two different X matrix rounding
            Xii_base = cfr_ldl(sdp_result['X'][ii], accuracy)
            Xii_base = vector(Xii_base.list())
            if correct_y:
                Xii_corr = cfr_ldl_corr(sdp_result['X'][ii], Zii, accuracy)
                Xii_corr = vector(Xii_corr.list())
            
            #update slacks
            for jj, mat in enumerate(table):
                #TODO: make this a sparse vector, as the table is sparse
                mvec = vector(mat.list())
                slack_base[jj] -= (Xii_base*mvec)
                if correct_y:
                    slack_corr[jj] -= (Xii_corr*mvec)
            if has_tqdm:
                pbar.set_description("{} is complete".format(ftype))
            else:
                print("{} is complete".format(ftype), flush=True)
        
        #
        # semidefinite effect on slacks is done
        # optimize the rest with LP
        # 
        #
        #
        # LP has form:
        #
        # linprog(c, A_ub=None, b_ub=None, A_eq=None, b_eq=None, 
        #         bounds=(0, None), method='highs', callback=None, 
        #         options=None, x0=None, integrality=None)
        #
        # minimize_x c * x
        # s.t. A_ub * x <= b_ub
        #      A_eq * x == b_eq
        #      bound[0] <= x <= bounds[1]
        # 
        # want to use method='highs-ds' for simplex

        print(best_y, "\n\n", matrix(slack_base).str(per), "\n\n", matrix(slack_corr).str(per))
        
        sdim = len(constraints_flags_vec)-2
        
        A_ub = matrix(QQ, [-onevec] + constraints_flags_vec[:sdim]).T
        b_ub = list(slack_corr) if correct_y else list(slack_base)
        bounds = [[None] + [0]*sdim, [None]*(1 + sdim)]
        c = [1] + [0]*sdim
        res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs-ds")
        
        print("Dual rounding done in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()
        return res

In [209]:
RED = []
NRD = []
for ii in range(len(MAT)):
    RED.append(cfr_ldl_corr(MAT[ii], KERN[ii]))
    NRD.append(cfr_ldl(MAT[ii]))

In [214]:
KERN[0]

[ 9/64  9/64 9/128     0     0 3/128]
[ 9/64  9/64 9/128     0     0 3/128]
[9/128 9/128 9/256     0     0 3/256]
[    0     0     0     0     0     0]
[    0     0     0     0     0     0]
[3/128 3/128 3/256     0     0 1/256]

In [166]:
def per(x):
    if len(str(x))<10:
        return str(x)
    else:
        return str(x.n(8))

In [191]:
G = GraphTheory
G.exclude(G(5))
optimize_problem(G, G(2), 5, certificate=True, rounding=4)

Ftypes constructed in 0.00s
Block sizes done in 0.01s
Block sizes are [6, 8, 8, 8, 8, -33, -2]
Calculating product matrices for 5 ftypes and 33 structures
Ftype on 3 points with edges=[[0, 1], [0, 2], [1, 2]] is complete: : 5it [00:00, 655.46it/s]
Table calculation done in 0.01s
Target and constraint calculation done in 0.00s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
SDP finished in 0.07s

Iter:  1 Ap: 1.00e+00 Pobj: -2.0783195e+01 Ad: 7.14e-01 Dobj:  3.5104979e-03 
Iter:  2 Ap: 1.00e+00 Pobj: -2.1231401e+01 Ad: 9.50e-01 Dobj: -4.1842059e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.7149451e+01 Ad: 9.22e-01 Dobj: -4.6029391e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -5.9133654e+00 Ad: 7.11e-01 Dobj: -4.6470501e-01 
Iter:  5 Ap: 5.72e-01 Pobj: -4.3931895e+00 Ad: 9.44e-01 Dobj: -4.6578852e-01 
Iter:  6 Ap: 9.96e-01 Pobj: -1.2255339e+00 Ad: 8.24e-01 Dobj: -4.8208190e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -9.0650662e-01 Ad: 6.29e-01 Dobj: -5.4665340e-01 
Iter:  8 Ap: 

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 0.7500000523184999
              x: [ 7.500e-01]
            nit: 0
          lower:  residual: [       inf]
                 marginals: [ 0.000e+00]
          upper:  residual: [       inf]
                 marginals: [ 0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  7.816e-02 ...  4.065e-01
                              5.918e-08]
                 marginals: [-1.000e+00 -0.000e+00 ... -0.000e+00
                             -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [16]:
# to check the block sizes
def get_block_sizes(self, target_size):
    theory = self
    flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
              for flag in theory.generate_flags(kk)]
    ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
              for flag in flags]
    block_sizes = [len(theory.generate_flags((target_size + ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    return block_sizes, constraints

In [199]:
def mval(matrix):
    ret = 0
    for xx in matrix.list():
        if ret<abs(xx):
            ret = abs(xx)
    return ret